# Project 2: Supervised Learning
### Building a Student Intervention System

## 1. Classification vs Regression

Your goal is to identify students who might need early intervention - which type of supervised machine learning problem is this, classification or regression? Why?

## 2. Exploring the Data

Let's go ahead and read in the student dataset first.

_To execute a code cell, click inside it and press **Shift+Enter**._

In [15]:
# Import libraries
import numpy as np
import pandas as pd
from sklearn import cross_validation
from IPython.display import display

In [16]:
# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"
# Note: The last column 'passed' is the target/label, all other are feature columns

Student data read successfully!



Now, can you find out the following facts about the dataset?
- Total number of students
- Number of students who passed
- Number of students who failed
- Graduation rate of the class (%)
- Number of features

_Use the code block below to compute these values. Instructions/steps are marked using **TODO**s._

## 3. Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Let's first separate our data into feature and target columns, and see if any features are non-numeric.<br/>
**Note**: For this dataset, the last column (`'passed'`) is the target or label we are trying to predict.

In [17]:
# TODO: Compute desired values - replace each '?' with an appropriate expression/function call
n_students = student_data.shape[0]
n_features = student_data.shape[1]-1
n_passed = list(student_data['passed']).count('yes')
n_failed = list(student_data['passed']).count('no')
grad_rate = float(n_passed)/n_students*100
print "Total number of students: {}".format(n_students)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Number of features: {}".format(n_features)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of students who passed: 265
Number of students who failed: 130
Number of features: 30
Graduation rate of the class: 67.09%


In [18]:
# Extract feature (X) and target (y) columns
feature_cols = list(student_data.columns[:-1])  # all columns but last are features
target_col = student_data.columns[-1]  # last column is the target/label
print "Feature column(s):-\n{}".format(feature_cols)
print "Target column: {}".format(target_col)

X_all = student_data[feature_cols]  # feature values for all students
y_all = student_data[target_col]  # corresponding targets/labels
print "\nFeature values:-"
print X_all.head()  # print the first 5 rows

Feature column(s):-
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']
Target column: passed

Feature values:-
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...    

### Preprocess feature columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation.

In [19]:
# Preprocess feature columns
def preprocess_features(X):
    outX = pd.DataFrame(index=X.index)  # output dataframe, initially empty

    # Check each column
    for col, col_data in X.iteritems():
        # If data type is non-numeric, try to replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])
        # Note: This should change the data type for yes/no columns to int

        # If still non-numeric, convert to one or more dummy variables
        if col_data.dtype == object:
            col_data = pd.get_dummies(col_data, prefix=col)  # e.g. 'school' => 'school_GP', 'school_MS'

        outX = outX.join(col_data)  # collect column(s) in output dataframe

    return outX

X_all = preprocess_features(X_all)
print "Processed feature columns ({}):-\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48):-
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Split data into training and test sets

So far, we have converted all _categorical_ features into numeric values. In this next step, we split the data (both features and corresponding labels) into training and test sets.

In [20]:
# First, decide how many training vs test samples you want
num_all = student_data.shape[0]  # same as len(student_data)
num_train = 300  # about 75% of the data
num_test = num_all - num_train


# TODO: Then, select features (X) and corresponding labels (y) for the training and test sets
# Note: Shuffle the data or randomly select samples to avoid any bias due to ordering in the dataset
X_train, X_test, y_train, y_test = cross_validation.train_test_split(X_all, y_all, test_size=num_test, 
                                                                             random_state=5)
print "Training set: {} samples".format(X_train.shape[0])
print "Test set: {} samples".format(X_test.shape[0])
# Note: If you need a validation set, extract it from within training data


Training set: 300 samples
Test set: 95 samples


## 4. Training and Evaluating Models
Choose 3 supervised learning models that are available in scikit-learn, and appropriate for this problem. For each model:

- What are the general applications of this model? What are its strengths and weaknesses?
- Given what you know about the data so far, why did you choose this model to apply?
- Fit this model to the training data, try to predict labels (for both training and test sets), and measure the F<sub>1</sub> score. Repeat this process with different training set sizes (100, 200, 300), keeping test set constant.

Produce a table showing training time, prediction time, F<sub>1</sub> score on training set and F<sub>1</sub> score on test set, for each training set size.

Note: You need to produce 3 such tables - one for each model.

In [21]:
# Train a model
import time

def train_classifier(clf, X_train, y_train):
    print "Training {}...".format(clf.__class__.__name__)
    start = time.time()
    clf.fit(X_train, y_train)
    end = time.time()
    print "Done!\nTraining time (secs): {:.3f}".format(end - start)

# TODO: Choose a model, import it and instantiate an object
from sklearn.svm import SVC
clf = SVC()

# Fit model to training data
train_classifier(clf, X_train, y_train)  # note: using entire training set here
#print clf  # you can inspect the learned model by printing it

Training SVC...
Done!
Training time (secs): 0.031


In [22]:
# Predict on training set and compute F1 score
from sklearn.metrics import f1_score


def predict_labels(clf, features, target):
    print "Predicting labels using {}...".format(clf.__class__.__name__)
    start = time.time()
    y_pred = clf.predict(features)
    end = time.time()
    print "Done!\nPrediction time (secs): {:.3f}".format(end - start)
    return f1_score(target.values, y_pred, pos_label='yes')

train_f1_score = predict_labels(clf, X_train, y_train)
print "F1 score for training set: {}".format(train_f1_score)

Predicting labels using SVC...
Done!
Prediction time (secs): 0.020
F1 score for training set: 0.876355748373


In [23]:
# Predict on test data
print "F1 score for test set: {}".format(predict_labels(clf, X_test, y_test))

Predicting labels using SVC...
Done!
Prediction time (secs): 0.008
F1 score for test set: 0.765957446809


In [24]:
#Generate tables of training/prediction times and training/test F1 scores.

#Same as above, but redone for the generation of a table without duplicate print statements or overly modifying given code.

def train_time(clf, X_train, y_train):    #returns training time
    start = time.time()
    clf.fit(X_train, y_train)
    end = time.time()
    return "{0:.3f}".format(end - start)

def predict_perf(clf, features, target):     #returns f1 score and prediction time
    start = time.time()
    y_pred = clf.predict(features)
    end = time.time()
    return f1_score(target.values, y_pred, pos_label='yes'), "{0:.3f}".format(end - start)




def training_table(clf, X_train, y_train, X_test, y_test):
    table = pd.DataFrame()
    sizes = [100, 200, 300]
    
    
    for s in sizes:
        time_train = train_time(clf, X_train[:s], y_train[:s])
        f1_train, time_train_pred = predict_perf(clf, X_train[:s], y_train[:s])
        f1_test, time_test = predict_perf(clf, X_test, y_test)
        row1 = pd.DataFrame({'a':["Training set size"], 'b':[s], 'c':["Training time (secs)"], 'd':[time_train]})
        row2 = pd.DataFrame({'a':["Training set size"], 'b':[s], 'c':["Prediction time (secs)"], 'd':[time_test]})
        row3 = pd.DataFrame({'a':["Training set size"], 'b':[s], 'c':["F1 score for training set"], 'd':[f1_train]})
        row4 = pd.DataFrame({'a':["Training set size"], 'b':[s], 'c':["F1 score for test set"], 'd':[f1_test]})
        table = table.append([row1, row2, row3, row4])

        
    table = table.pivot_table(index='c', columns=['a','b'], values='d', aggfunc=lambda x:x)
    table = table.reindex(index=table.index[::-1])
    table.index.name = None
    table.columns.names = [None, None]
    print("\n"+format(clf.__class__.__name__))
    display(table)

In [25]:
# Train and predict using different training set sizes

def train_predict(clf, X_train, y_train, X_test, y_test):
    print "------------------------------------------"
    print "Training set size: {}".format(len(X_train))
    train_classifier(clf, X_train, y_train)
    print "F1 score for training set: {}".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {}".format(predict_labels(clf, X_test, y_test))

# TODO: Run the helper function above for desired subsets of training data
# Note: Keep the test set constant

train_predict(clf, X_train[:100], y_train[:100], X_test, y_test)        
                                                                                 
train_predict(clf, X_train[:200], y_train[:200], X_test, y_test)
                                                                                 
train_predict(clf, X_train, y_train, X_test, y_test)  


training_table(clf, X_train, y_train, X_test, y_test)
                                                                                 


------------------------------------------
Training set size: 100
Training SVC...
Done!
Training time (secs): 0.016
Predicting labels using SVC...
Done!
Prediction time (secs): 0.004
F1 score for training set: 0.851851851852
Predicting labels using SVC...
Done!
Prediction time (secs): 0.004
F1 score for test set: 0.789808917197
------------------------------------------
Training set size: 200
Training SVC...
Done!
Training time (secs): 0.004
Predicting labels using SVC...
Done!
Prediction time (secs): 0.016
F1 score for training set: 0.861635220126
Predicting labels using SVC...
Done!
Prediction time (secs): 0.000
F1 score for test set: 0.786666666667
------------------------------------------
Training set size: 300
Training SVC...
Done!
Training time (secs): 0.019
Predicting labels using SVC...
Done!
Prediction time (secs): 0.009
F1 score for training set: 0.876355748373
Predicting labels using SVC...
Done!
Prediction time (secs): 0.002
F1 score for test set: 0.765957446809

SVC


Training set size                    
                                        100       200       300
Training time (secs)                  0.004     0.004     0.008
Prediction time (secs)                0.000     0.000     0.004
F1 score for training set          0.851852  0.861635  0.876356
F1 score for test set              0.789809  0.786667  0.765957

In [26]:
# TODO: Train and predict using two other models

from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()
train_classifier(clf, X_train, y_train)
print "\n"
train_f1_score = predict_labels(clf, X_train, y_train)
print "\n"
print "F1 score for training set: {}".format(train_f1_score)
print "F1 score for test set: {}".format(predict_labels(clf, X_test, y_test))

train_predict(clf, X_train[:100], y_train[:100], X_test, y_test)        
                                                                                 
train_predict(clf, X_train[:200], y_train[:200], X_test, y_test)
                                                                                 
train_predict(clf, X_train, y_train, X_test, y_test)     

training_table(clf, X_train, y_train, X_test, y_test)

Training GaussianNB...
Done!
Training time (secs): 0.000


Predicting labels using GaussianNB...
Done!
Prediction time (secs): 0.005


F1 score for training set: 0.827423167849
Predicting labels using GaussianNB...
Done!
Prediction time (secs): 0.000
F1 score for test set: 0.766917293233
------------------------------------------
Training set size: 100
Training GaussianNB...
Done!
Training time (secs): 0.000
Predicting labels using GaussianNB...
Done!
Prediction time (secs): 0.001
F1 score for training set: 0.844720496894
Predicting labels using GaussianNB...
Done!
Prediction time (secs): 0.000
F1 score for test set: 0.763888888889
------------------------------------------
Training set size: 200
Training GaussianNB...
Done!
Training time (secs): 0.001
Predicting labels using GaussianNB...
Done!
Prediction time (secs): 0.001
F1 score for training set: 0.830985915493
Predicting labels using GaussianNB...
Done!
Prediction time (secs): 0.000
F1 score for test set: 0.713178294574
---------

Training set size                    
                                        100       200       300
Training time (secs)                  0.000     0.003     0.000
Prediction time (secs)                0.000     0.000     0.004
F1 score for training set           0.84472  0.830986  0.827423
F1 score for test set              0.763889  0.713178  0.766917

In [27]:
from sklearn.neighbors import KNeighborsClassifier
clf = KNeighborsClassifier(n_neighbors=5)
train_classifier(clf, X_train, y_train)
print "\n"
train_f1_score = predict_labels(clf, X_train, y_train)
print "\n"
print "F1 score for training set: {}".format(train_f1_score)
print "F1 score for test set: {}".format(predict_labels(clf, X_test, y_test))

train_predict(clf, X_train[:100], y_train[:100], X_test, y_test)        
                                                                                 
train_predict(clf, X_train[:200], y_train[:200], X_test, y_test)
                                                                                 
train_predict(clf, X_train, y_train, X_test, y_test)  

training_table(clf, X_train, y_train, X_test, y_test)

Training KNeighborsClassifier...
Done!
Training time (secs): 0.000


Predicting labels using KNeighborsClassifier...
Done!
Prediction time (secs): 0.009


F1 score for training set: 0.851351351351
Predicting labels using KNeighborsClassifier...
Done!
Prediction time (secs): 0.004
F1 score for test set: 0.768115942029
------------------------------------------
Training set size: 100
Training KNeighborsClassifier...
Done!
Training time (secs): 0.000
Predicting labels using KNeighborsClassifier...
Done!
Prediction time (secs): 0.002
F1 score for training set: 0.855263157895
Predicting labels using KNeighborsClassifier...
Done!
Prediction time (secs): 0.003
F1 score for test set: 0.791666666667
------------------------------------------
Training set size: 200
Training KNeighborsClassifier...
Done!
Training time (secs): 0.001
Predicting labels using KNeighborsClassifier...
Done!
Prediction time (secs): 0.001
F1 score for training set: 0.842809364548
Predicting labels using KNeighborsClassif

Training set size                    
                                        100       200       300
Training time (secs)                  0.004     0.004     0.000
Prediction time (secs)                0.004     0.004     0.004
F1 score for training set          0.855263  0.842809  0.851351
F1 score for test set              0.791667  0.780822  0.768116

## 5. Choosing the Best Model

- Based on the experiments you performed earlier, in 1-2 paragraphs explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?
- In 1-2 paragraphs explain to the board of supervisors in layman's terms how the final model chosen is supposed to work (for example if you chose a Decision Tree or Support Vector Machine, how does it make a prediction).
- Fine-tune the model. Use Gridsearch with at least one important parameter tuned and with at least 3 settings. Use the entire training set for this.
- What is the model's final F<sub>1</sub> score?

In [28]:
# TODO: Fine-tune your model and report the best F1 score
from sklearn.grid_search import GridSearchCV
from sklearn import metrics

def performance_metric(target, pred):
    return f1_score(target.values, pred, pos_label='yes')
 

svc = SVC()
params = {'kernel':['poly', 'rbf', 'sigmoid'], 'degree':[1, 2, 3], 'C':[0.1, 1, 10]}
scorer = metrics.make_scorer(performance_metric, greater_is_better=True)

clf = GridSearchCV(svc, params, scoring=scorer)
clf.fit(X_train, y_train)


print "Best F1 score on left out training data: " + str(clf.best_score_)
print "F1 score for test set: " + str(clf.score(X_test, y_test))
print "Best params: " + str(clf.best_params_)


Best F1 score on left out training data: 0.809959158282
F1 score for test set: 0.802721088435
Best params: {'kernel': 'poly', 'C': 1, 'degree': 1}
